# Unit Testing in Python

## Part 1: Why We Need Unit Tests and Their Importance

Unit tests are a type of software testing where individual units or components of the software are tested. The purpose is to validate that each unit of the software performs as expected. A unit is the smallest testable part of any software, typically a function or a method.

1. **Improves Code Quality**: By writing unit tests, developers can identify and fix bugs early in the development cycle, which leads to more stable and reliable software.

2. **Facilitates Refactoring**: Unit tests ensure that code changes do not introduce new bugs. Developers can refactor code with confidence, knowing that tests will catch any regressions.

3. **Documentation**: Tests can serve as documentation for the code. They describe how the code is supposed to behave and can help new developers understand the codebase.

## Part 2: The `unittest` Module

Python’s `unittest` module, inspired by JUnit, is a built-in module used to create and run unit tests. It supports test automation, sharing of setup and shutdown code for tests, aggregation of tests into collections, and independence of the tests from the reporting framework.

### Base Features of `unittest`:

- **Test Case**: The smallest unit of testing. It checks for a specific response to a particular set of inputs.

- **Test Suite**: A collection of test cases, test suites, or both.

- **Test Runner**: A component that orchestrates the execution of tests and provides the outcome to the user.

- **Test Fixtures**: Resources needed for a test, such as temporary databases or files, which need to be set up before the test runs and torn down afterward.

- Mocking: Replacing parts of the system under test with mock objects to isolate the behavior of the test.

### Example: Banking System Code

First, let's define a simple banking system class.

Now, let's write the `unittest` code with `setUp`, `tearDown`, and a fixture.

In [17]:
# test_banking_system_unittest.py

import unittest
from modules.banking_system import BankAccount

def create_account_with_balance(balance):
    """Fixture function to create a BankAccount with a given balance."""
    account = BankAccount(balance)
    return account

class TestBankAccount(unittest.TestCase):
    def setUp(self):
        """Set up test fixtures before each test method."""
        self.account = BankAccount()
        self.account.deposit(100)  # Start each test with a balance of 100
    
    def tearDown(self):
        """Tear down test fixtures after each test method."""
        del self.account
    
    def test_deposit(self):
        self.account.deposit(50)
        self.assertEqual(self.account.get_balance(), 150)
    
    def test_withdraw(self):
        self.account.withdraw(30)
        self.assertEqual(self.account.get_balance(), 70)
    
    def test_withdraw_insufficient_funds(self):
        with self.assertRaises(ValueError) as context:
            self.account.withdraw(150)
        self.assertEqual(str(context.exception), "Insufficient funds")
    
    def test_deposit_invalid_amount(self):
        with self.assertRaises(ValueError) as context:
            self.account.deposit(-20)
        self.assertEqual(str(context.exception), "Deposit amount must be positive")
    
    def test_withdraw_invalid_amount(self):
        with self.assertRaises(ValueError) as context:
            self.account.withdraw(-10)
        self.assertEqual(str(context.exception), "Withdraw amount must be positive")

    def test_fixture_account(self):
        account = create_account_with_balance(200)
        self.assertEqual(account.get_balance(), 200)

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

......
----------------------------------------------------------------------
Ran 6 tests in 0.006s

OK


- **`setUp` Method:** Initializes a `BankAccount` object and deposits 100 units before each test.
- **`tearDown` Method:** Cleans up by deleting the `BankAccount` object after each test.
- **Fixture:** The `create_account_with_balance` function is used as a fixture to create `BankAccount` objects with a specific balance. This allows us to reuse the same `BankAccount` object across multiple tests without having to create new objects for each test.

#### Example: Mocking

**Mocking** is a technique in unit testing where you create mock objects to simulate the behavior of real objects. This is particularly useful when you want to isolate the part of the code you are testing from external dependencies, such as databases, web services, or other complex systems. By using mocks, you can control the environment in which your tests run, ensuring that they are reliable and repeatable.

**Why is Mocking Used?**
- **Isolation:** It allows you to test a unit of code in isolation by replacing external dependencies with mock objects.
- **Control:** You can control the behavior of the mock objects to test different scenarios, including edge cases and error conditions.

**Mocking Example with Unittest**
In the above we have an example where we mock a web service call in a banking system to fetch exchange rates. Now, let's write the `unittest` code that mocks the web service call.

In [2]:
# test_banking_system_mock.py

import unittest
from unittest.mock import patch
from modules.banking_system import BankAccount

class TestBankAccount(unittest.TestCase):
    def setUp(self):
        self.account = BankAccount()
        self.account.deposit(100)
    
    def tearDown(self):
        del self.account
    
    @patch('modules.banking_system.requests.get')
    def test_convert_currency(self, mock_get):
        mock_response = {
            'rates': {
                'EUR': 0.85
            }
        }
        mock_get.return_value.status_code = 200
        mock_get.return_value.json.return_value = mock_response

        result = self.account.convert_currency(100, 'EUR')
        self.assertEqual(result, 85)
    
    @patch('modules.banking_system.requests.get')
    def test_convert_currency_api_failure(self, mock_get):
        mock_get.return_value.status_code = 500
        
        with self.assertRaises(Exception) as context:
            self.account.convert_currency(100, 'EUR')
        self.assertEqual(str(context.exception), "Error fetching exchange rate")

if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)


..
----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


- **Mocking the Web Service:** The `@patch('banking_system.requests.get')` decorator replaces the `requests.get` method with a mock object during the test.
- **Mock Response:** We define a mock response for the exchange rate API to simulate different scenarios.
  - In `test_convert_currency`, the mock response simulates a successful API call returning an exchange rate of 0.85 for EUR.
  - In `test_convert_currency_api_failure`, the mock response simulates an API failure with a status code of 500.
- **Assertions:** The tests check that the `convert_currency` method correctly converts the amount using the mocked exchange rate and raises an exception on API failure.

## Part 3: The `pytest` Package in Python

`pytest` is a powerful testing framework for Python that makes it easy to write simple and scalable test cases.


### Base Features of `pytest`:

- **Simple Syntax**: Tests are written as regular functions with assert statements.

- **Fixtures**: Reusable setup and teardown code for tests.

- **Parameterization**: Running a test with multiple sets of data.

- **Plugins**: Extending pytest functionality with plugins.

### Example

Let's assume we have a simple e-commerce system with functionalities like adding products to a cart, checking out, and applying discounts. Please look at: `modules/ecommerce.py`. 

We'll create a series of tests to validate these functionalities. 

First, ensure you have pytest and pytest-mock installed:

In [14]:
! pip install pytest pytest-mock ipytest -q

In [15]:
import ipytest
ipytest.autoconfig()

In [16]:
%%ipytest

from modules.ecommerce import Product, Cart, PaymentProcessor

@pytest.fixture
def cart():
    return Cart()

@pytest.fixture
def products():
    return [
        Product("Product 1", 100),
        Product("Product 2", 200),
        Product("Product 3", 300)
    ]

def test_add_product(cart, products):
    cart.add_product(products[0])
    assert len(cart.products) == 1
    cart.add_product(products[1])
    assert len(cart.products) == 2

@pytest.mark.parametrize(
    "products, expected_total",
    [
        ([Product("Product 1", 100), Product("Product 2", 200)], 300),
        ([Product("Product 3", 300)], 300),
        ([Product("Product 1", 100), Product("Product 2", 200), Product("Product 3", 300)], 600),
    ]
)
def test_cart_total(cart, products, expected_total):
    for product in products:
        cart.add_product(product)
    assert cart.total() == expected_total

@pytest.mark.skip(reason="Skipping discount test, discount feature not yet finalized")
def test_apply_discount(cart, products):
    for product in products:
        cart.add_product(product)
    total_with_discount = cart.apply_discount(10)
    assert total_with_discount == cart.total() * 0.9

def test_payment_processor_success(mocker):
    processor = PaymentProcessor()
    mocker.patch.object(processor, 'process_payment', return_value=True)
    assert processor.process_payment(100) == True

def test_payment_processor_failure(mocker):
    processor = PaymentProcessor()
    mocker.patch.object(processor, 'process_payment', side_effect=ValueError("Amount must be greater than zero"))
    with pytest.raises(ValueError, match="Amount must be greater than zero"):
        processor.process_payment(0)

....s..                                                                                      [100%]
6 passed, 1 skipped in 0.09s


In this example:

- **Fixtures** (`cart` and `products`) are used to set up common objects used in multiple tests.
- **Parameterization** is used in the `test_cart_total` test to run the test with different sets of products and expected totals.
- **Skipping tests** is demonstrated with the `test_apply_discount` test, which is skipped with a reason provided.
- **Mocking** is used in the `test_payment_processor_success` and `test_payment_processor_failure` tests to simulate the behavior of the `process_payment` method.
- **Plugin support** To add plugin support, we can use pytest plugins. For example, `pytest-xdist` for parallel test execution. To run tests with parallel execution: `pytest -n 4` (where 4 is the number of CPU cores to use)

## Part 4: Comparison of `unittest` and `pytest`

Here's a detailed comparison of the core differences between PyTest and Unittest, highlighting their features and capabilities.

<table>
  <tr>
    <th>Features</th>
    <th>PyTest</th>
    <th>Unittest</th>
  </tr>
  <tr>
    <td>Test Discovery</td>
    <td>Automatic test discovery, finds and runs tests without boilerplate</td>
    <td>Requires manual test discovery by explicitly defining test cases</td>
  </tr>
  <tr>
    <td>Fixture Support</td>
    <td>Powerful and flexible fixture support</td>
    <td>Limited fixture support, mainly through the setup and teardown methods</td>
  </tr>
  <tr>
    <td>Test Execution</td>
    <td>Supports parallel test execution, faster runtime</td>
    <td>Sequential test execution, one test at a time</td>
  </tr>
  <tr>
    <td>Test Execution Options</td>
    <td>Provides various options for test execution customization</td>
    <td>Offers fewer options for customizing the test execution process</td>
  </tr>
  <tr>
    <td>Assertion Methods</td>
    <td>Rich set of built-in assertion methods</td>
    <td>Standard assertion methods provided by the unittest module</td>
  </tr>
  <tr>
    <td>Test Organization</td>
    <td>Test functions can be organized in a flexible manner</td>
    <td>Test cases are organized as classes, providing a more structured approach</td>
  </tr>
  <tr>
    <td>Skipping Tests</td>
    <td>Built-in mechanism for skipping tests</td>
    <td>Ability to skip tests using decorators or conditional statements</td>
  </tr>
  <tr>
    <td>Test Parameterization</td>
    <td>Built-in support for parameterized tests</td>
    <td>Parameterization can be achieved using decorators or conditional logic</td>
  </tr>
  <tr>
    <td>Plugin Ecosystem</td>
    <td>Large and active plugin ecosystem with many useful plugins</td>
    <td>Limited plugin support, fewer third-party extensions available</td>
  </tr>
  <tr>
    <td>Output Readability</td>
    <td>Detailed and readable output for failed tests</td>
    <td>Basic output with less detailed information</td>
  </tr>
</table>
